# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)
print(cities)

['amderma', 'severo-kurilsk', 'kapaa', 'bambous virieux', 'punta arenas', 'hilo', 'jamestown', 'nishihara', 'namibe', 'nagornyy', 'port elizabeth', 'sao filipe', 'castro', 'sioux lookout', 'saint-augustin', 'nioro', 'hermanus', 'atuona', 'saint-pierre', 'port alfred', 'vaitupu', 'bell ville', 'port blair', 'bluff', 'isangel', 'airai', 'bengkulu', 'new norfolk', 'bredasdorp', 'poum', 'lebu', 'ushuaia', 'arman', 'rikitea', 'nyurba', 'kaitangata', 'touros', 'barrow', 'killarney', 'illoqqortoormiut', 'avera', 'mataura', 'sola', 'sinjah', 'dikson', 'puerto suarez', 'busselton', 'carnarvon', 'mweka', 'tsihombe', 'gweta', 'meyungs', 'tazovskiy', 'guerrero negro', 'chuy', 'thompson', 'ukiah', 'nizhneyansk', 'kotlyarevskaya', 'alekseyevka', 'groningen', 'sisimiut', 'anadyr', 'komsomolskiy', 'ayan', 'severnyy', 'alvorada', 'vaini', 'araioses', 'neyshabur', 'nyaunglebin', 'sataua', 'kruisfontein', 'chicama', 'red bluff', 'altamont', 'cabo san lucas', 'buala', 'flin flon', 'arraial do cabo', 'husa

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [40]:
# Create a dictionary to hold each city name as a key
# and a list of city weather data as values

all_cities_weather = {}


# Loop through the list of cities and append them to the 
for city in cities[1:5]:

    # Create a list to hold weather data for a single city
    city_weather = []
    city_weather_dict = {}

    # Save config information
    url = "http://api.openweathermap.org/data/2.5/weather?"

    # Build query URL
    query_url = f"{url}appid={weather_api_key}&q={city}&units=Imperial"

    # Read JSON data
    weather_response = requests.get(query_url)
    weather_json = weather_response.json()

    # Set variables to hold relevant data
    city_name = weather_json["name"]
    lat = weather_json["coord"]["lat"]
    lon = weather_json["coord"]["lon"]
    max_temp = weather_json["main"]["temp_max"]
    cloud = weather_json["clouds"]["all"]
    wind_speed = weather_json["wind"]["speed"]
    humidity = weather_json["main"]["humidity"]
    date = weather_json["dt"]
    id_code = weather_json["id"]
    country = weather_json["sys"]["country"]

    # Append variables to list
    city_weather.append(city_name)
    city_weather.append(cloud)
    city_weather.append(country)
    city_weather.append(date)
    city_weather.append(humidity)
    city_weather.append(lat)
    city_weather.append(lon)
    city_weather.append(max_temp)
    city_weather.append(wind_speed)

    # Update all_cities_weather dictionary to include data for that city
    all_cities_weather.update({city_name:city_weather})

    # Delay 1.5 seconds between API calls
    time.sleep(1.5)
# try:
# except KeyError:
#     print("City not found. Skipping...")
        
print(all_cities_weather)
    


{"Severo-Kuril'sk": ["Severo-Kuril'sk", 33, 'RU', 1584234285, 76, 50.68, 156.12, 20.98, 6.64], 'Kapaa': ['Kapaa', 1, 'US', 1584234462, 57, 22.08, -159.32, 80.6, 16.11], 'Bambous Virieux': ['Bambous Virieux', 40, 'MU', 1584234220, 88, -20.34, 57.76, 77, 2.24], 'Punta Arenas': ['Punta Arenas', 100, 'CL', 1584234465, 76, -53.15, -70.92, 57, 25.28]}


In [6]:
print(json.dumps(weather_json, indent=4))

{
    "coord": {
        "lon": -70.92,
        "lat": -53.15
    },
    "weather": [
        {
            "id": 500,
            "main": "Rain",
            "description": "light rain",
            "icon": "10n"
        }
    ],
    "base": "stations",
    "main": {
        "temp": 55.22,
        "feels_like": 40.48,
        "temp_min": 55,
        "temp_max": 55.4,
        "pressure": 1005,
        "humidity": 76
    },
    "visibility": 10000,
    "wind": {
        "speed": 25.28,
        "deg": 300
    },
    "rain": {
        "3h": 0.44
    },
    "clouds": {
        "all": 100
    },
    "dt": 1584232502,
    "sys": {
        "type": 1,
        "id": 8487,
        "country": "CL",
        "sunrise": 1584182106,
        "sunset": 1584227429
    },
    "timezone": -10800,
    "id": 3874787,
    "name": "Punta Arenas",
    "cod": 200
}


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [37]:
Summary = pd.DataFrame.from_dict(all_cities_weather, orient="index")

Summary = Summary.rename(columns=
                        {0:"City",
                        1:"Cloudiness",
                        2:"Country",
                        3:"Date",
                        4:"Humidity",
                        5:"Lat",
                        6:"Lng",
                        7:"Max Temp",
                        8:"Wind Speed"}
                        )
Summary
Summary.reset_index(drop=True)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Severo-Kuril'sk,33,RU,1584234285,76,50.68,156.12,20.98,6.64
1,Kapaa,1,US,1584234462,57,22.08,-159.32,80.60,16.11
2,Bambous Virieux,40,MU,1584234220,88,-20.34,57.76,77.00,2.24
3,Punta Arenas,100,CL,1584234465,76,-53.15,-70.92,57.00,25.28


### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots


In [ ]:
# Create Northern and Southern Hemisphere DataFrames


####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression